# Proposal to adjust commanded MAXMAG based on search hits constraint

The MAXMAG value is part of the ACA star catalog that gets uplinked to the
OBC and subsequently sent as part of the ACA search command(s) from the OBC
to the ACA PEA during star acquisition.

MAXMAG is used by the PEA in two ways

- For acquisition, it sets the acceptance criterion for search hits to be
  considered as candidates for the acq star being searched. A higher value
  of MAXMAG means more spurious hits due to CCD dark current non-uniformity
  will be accepted. 
  
  It can (and does) happen that the fixed buffer of 50 search hits
  per readout can be filled, leading to the intended star candidate
  not being found. This can happen regardless of star magnitude.

- For guide, it sets the faint threshold at which the PEA drops track of a
  star.


## Calibrating MAXMAG limit that saturates the 50 search hit buffer

Mark Baski used ASVT to empirically calibrate the value of MAXMAG at
which the 50 hit buffer fills, as a function of CCD temperature and search box
size.

https://occweb.cfa.harvard.edu/twiki/bin/view/Aspect/PeaMaxMagTesting

<img src="PEA_MAXMAG_contours.png" style="width:600px;">

## Code

https://github.com/sot/proseco/pull/376

In [1]:
import warnings
import numpy as np
from proseco.acq import MAXMAGS
import proseco.characteristics as ACA

## Getting MAXMAG search hits constraint from ASVT data

The data from the contour plot that Mark Baski generated was translated into
a structure that allows for convenient interpolation for an arbitrary CCD 
temperature and allowed search box size (60, 80, 100, ... 240) arcsec.

In this section we show the code and some results.

In [2]:
def get_maxmag(box_size: int, t_ccd: float) -> float:
    """
    Get maxmag for given box_size and t_ccd.

    This corresponds to the MAXMAG that results in exactly 50 search hits. See
    https://occweb.cfa.harvard.edu/twiki/bin/view/Aspect/PeaMaxMagTesting.

    :param box_size: box size (int, arcsec)
    :param t_ccd: CCD temperature (float, C)
    :returns: maxmag (float)
    """
    if t_ccd < -10.0:
        return 11.2
    if t_ccd > 0:
        warnings.warn(f"Clipping {t_ccd=} to 0.0 for interpolating MAXMAGs table")
        t_ccd = 0.0

    if box_size not in MAXMAGS:
        raise ValueError(
            f"illegal value of box_size={box_size}, must be 60, 80, 100, .., 240"
        )
    maxmag = np.interp(t_ccd, xp=MAXMAGS["t_ccds"], fp=MAXMAGS[box_size])

    # Need to round to nearest 0.01 mag because of values that are just slightly
    # below 11.2. These need to become exactly 11.2.
    return maxmag.round(2)


In [3]:
print(140, -4.0, get_maxmag(box_size=140, t_ccd=-4))
print(140, -12.0, get_maxmag(box_size=140, t_ccd=-12))
print(220, -10.0, get_maxmag(box_size=220, t_ccd=-10))


140 -4.0 10.79
140 -12.0 11.2
220 -10.0 11.15


In [4]:
print(60, 2.0, get_maxmag(box_size=60, t_ccd=2.0))


60 2.0 10.9


/var/folders/zn/910d7qgd1ydd4bvww62b6b9r0000gp/T/ipykernel_97242/3923574408.py:15: UserWarning: Clipping t_ccd=2.0 to 0.0 for interpolating MAXMAGs table
  warnings.warn(f"Clipping {t_ccd=} to 0.0 for interpolating MAXMAGs table")


In [5]:
try:
    get_maxmag(box_size=70, t_ccd=-2.0)
except ValueError as err:
    print(err)


illegal value of box_size=70, must be 60, 80, 100, .., 240


## Filtering the list of box sizes during acquisition star selection

The acquisition star selection process starts with a list of possible search
box sizes. This function filters that list to exclude any box sizes that would
violate the constraint of limiting the number of search hits less than 50.

In [6]:
def filter_box_sizes_for_maxmag(
    mag: float, mag_err: float, box_sizes: np.ndarray, t_ccd: float
) -> np.ndarray:
    """Filter the list of box sizes

    First compute the smallest allowed value of MAXMAG for this star, which is
    the star mag + 3 times the star mag error (clipped to be within 0.5 to 1.5
    mag, nominal).

    For each box size and t_ccd compute the MAXMAG that keeps the search hits
    at exactly 50. Then keep the box sizes where MAXMAG is less than the star
    minimum MAXMAG.

    :param mag: star mag (float)
    :param mag_err: star mag error (float)
    :param box_sizes: ndarray of box sizes (float, arcsec)
    :param t_ccd: CCD temperature (float, C)
    :returns: ndarray of box sizes (float, arcsec)
    """
    maxmag_min = mag + np.clip(
        2 * mag_err, 
        a_min=ACA.min_delta_maxmag,  # 0.5 mag
        a_max=ACA.max_delta_maxmag)  # 1.5 mag
    print(f"{mag=}")
    print(f"{mag_err=}")
    print(f"{t_ccd=:.2f}")
    print(f"{2 * mag_err=:.2f}")
    print(f"{ACA.min_delta_maxmag=}, {ACA.max_delta_maxmag=}")
    print(f"Initial {maxmag_min=:.2f}")

    # Hard limit of ACA.max_maxmag (11.2) from operational change made in 2019.
    # We always accept a maxmag of 11.2 regardless of star mag / mag_err.
    # Starcheck will warn if the mag is too close to maxmag.
    maxmag_min = maxmag_min.clip(None, ACA.max_maxmag)
    print(f"After clipping to 11.2 {maxmag_min=:.2f}")

    for box_size in box_sizes:
        print(f"{box_size=} {get_maxmag(box_size, t_ccd)=:.2f}")
    ok = [maxmag_min <= get_maxmag(box_size, t_ccd) for box_size in box_sizes]
    out = box_sizes[ok]  # type: np.ndarray

    # Always allow at least the smallest box size. This situation will be
    # flagged in ACA review.
    if len(out) == 0:
        out = np.array([60], dtype=np.int64)

    return out

In [7]:
box_sizes = np.array([60, 80, 100, 120, 140, 160])

In [8]:
filter_box_sizes_for_maxmag(mag=10.2, mag_err=0.3, box_sizes=box_sizes, t_ccd=-6.0)

mag=10.2
mag_err=0.3
t_ccd=-6.00
2 * mag_err=0.60
ACA.min_delta_maxmag=0.5, ACA.max_delta_maxmag=1.5
Initial maxmag_min=10.80
After clipping to 11.2 maxmag_min=10.80
box_size=60 get_maxmag(box_size, t_ccd)=11.20
box_size=80 get_maxmag(box_size, t_ccd)=11.20
box_size=100 get_maxmag(box_size, t_ccd)=11.15
box_size=120 get_maxmag(box_size, t_ccd)=11.04
box_size=140 get_maxmag(box_size, t_ccd)=10.97
box_size=160 get_maxmag(box_size, t_ccd)=10.92


array([ 60,  80, 100, 120, 140, 160])

In [9]:
filter_box_sizes_for_maxmag(mag=10.2, mag_err=0.4, box_sizes=box_sizes, t_ccd=-6.0)

mag=10.2
mag_err=0.4
t_ccd=-6.00
2 * mag_err=0.80
ACA.min_delta_maxmag=0.5, ACA.max_delta_maxmag=1.5
Initial maxmag_min=11.00
After clipping to 11.2 maxmag_min=11.00
box_size=60 get_maxmag(box_size, t_ccd)=11.20
box_size=80 get_maxmag(box_size, t_ccd)=11.20
box_size=100 get_maxmag(box_size, t_ccd)=11.15
box_size=120 get_maxmag(box_size, t_ccd)=11.04
box_size=140 get_maxmag(box_size, t_ccd)=10.97
box_size=160 get_maxmag(box_size, t_ccd)=10.92


array([ 60,  80, 100, 120])

In [10]:
filter_box_sizes_for_maxmag(mag=10.5, mag_err=0.05, box_sizes=box_sizes, t_ccd=-6.0)

mag=10.5
mag_err=0.05
t_ccd=-6.00
2 * mag_err=0.10
ACA.min_delta_maxmag=0.5, ACA.max_delta_maxmag=1.5
Initial maxmag_min=11.00
After clipping to 11.2 maxmag_min=11.00
box_size=60 get_maxmag(box_size, t_ccd)=11.20
box_size=80 get_maxmag(box_size, t_ccd)=11.20
box_size=100 get_maxmag(box_size, t_ccd)=11.15
box_size=120 get_maxmag(box_size, t_ccd)=11.04
box_size=140 get_maxmag(box_size, t_ccd)=10.97
box_size=160 get_maxmag(box_size, t_ccd)=10.92


array([ 60,  80, 100, 120])

In [11]:
filter_box_sizes_for_maxmag(mag=10.9, mag_err=0.5, box_sizes=box_sizes, t_ccd=-1.0)

mag=10.9
mag_err=0.5
t_ccd=-1.00
2 * mag_err=1.00
ACA.min_delta_maxmag=0.5, ACA.max_delta_maxmag=1.5
Initial maxmag_min=11.90
After clipping to 11.2 maxmag_min=11.20
box_size=60 get_maxmag(box_size, t_ccd)=10.97
box_size=80 get_maxmag(box_size, t_ccd)=10.80
box_size=100 get_maxmag(box_size, t_ccd)=10.61
box_size=120 get_maxmag(box_size, t_ccd)=10.55
box_size=140 get_maxmag(box_size, t_ccd)=10.50
box_size=160 get_maxmag(box_size, t_ccd)=10.46


array([60])

## Generating the final commanded catalog MAXMAG value

In [12]:
def get_maxmag_cmd(mag, box_size, t_ccd):
    maxmag_legacy = mag + ACA.max_delta_maxmag  # Legacy MAG + 1.5
    print(f"Unclipped {maxmag_legacy=}")
    
    maxmag_legacy = np.clip(maxmag_legacy, None, ACA.max_maxmag)  # Clip to 11.2
    maxmag_search_hits = get_maxmag(box_size, t_ccd)  # Search hits < 50 limit
    print(f"Clipped {maxmag_legacy=}")
    print(f"{maxmag_search_hits=}")
    
    maxmag_cmd = min(maxmag_legacy, maxmag_search_hits)
    print(f"{maxmag_cmd=}")


In [13]:
get_maxmag_cmd(mag=10.2, box_size=140, t_ccd=-6.0)

Unclipped maxmag_legacy=11.7
Clipped maxmag_legacy=11.2
maxmag_search_hits=10.97
maxmag_cmd=10.97


In [14]:
get_maxmag_cmd(mag=9.2, box_size=140, t_ccd=-6.0)

Unclipped maxmag_legacy=10.7
Clipped maxmag_legacy=10.7
maxmag_search_hits=10.97
maxmag_cmd=10.7


In [15]:
get_maxmag_cmd(mag=10.9, box_size=60, t_ccd=-1.0)

Unclipped maxmag_legacy=12.4
Clipped maxmag_legacy=11.2
maxmag_search_hits=10.97
maxmag_cmd=10.97
